# 2024-01-10 17 - Estimating more precise constraint levels -- PatchTST.ipynb
---
Fork of NB 13 to get the epsilons for patch tst on electricity, weather, exchange per `(dataset,model,pred_len)`

## Fetching runs

In [2]:
import wandb
from math import isnan 
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import wandb
from tqdm import tqdm
# from cycler import cycler
import matplotlib as mpl
from IPython.display import Markdown, display
    
api = wandb.Api()
project = "Autoformer"
workspace = "alelab"

# experiment_tags = [
#     "e1_weather_10e_statinformed",
#     "e2_electricity_10e_statinformed",
#     "e3_weather_10e_statinformed_seeds23",
#     "e4_electricity_10e_statinformed_seeds23"
#     ]

experiment_tags = ["e7_patchtst_erm_seeds123"]

# get all runs that both: 1.  match any experiment tag and 2. are finished
runs = api.runs(f"{workspace}/{project}",
                {"$and": [
                    {"tags": {"$in": experiment_tags}},
                    # Get only ERM runs: {"config.dual_lr": 0},
                    {"config.dual_lr": 0},
                    {"state": "finished"}
                ]})

def tag_experiment(run):
    for tag in experiment_tags:
        if tag in run.tags:
            return tag
    return ''

def runs_into_df(runs):
    all_runs = []
    run_counter = 0
    for run in tqdm(runs):
        run_counter += 1
        for split in ["train", "test","val"]:
            for metric in ["mse",]:
                pred_len = run.config["pred_len"]
                metrics = np.zeros(pred_len)
                for i in range(pred_len):
                    run_dict = {**run.config}
                    #run_dict["constraint_level"] = constraint_level
                    run_dict[f"{metric}"] = run.summary[f"{metric}/{split}/{i}"]
                    #run_dict[f"{metric}"] = run.summary.get(f"{metric}/{split}/{i}",run.summary.get(f"mse/{split}/{i}",np.nan)) #god forgive me for this line
                    run_dict["step"]=i
                    run_dict["epoch"]=run.summary["epoch"]
                    run_dict["infeasible_rate"]=run.summary[f"infeasible_rate/{split}"]
                    run_dict["infeasibles"]=run.summary[f"infeasibles/{split}"]
                    run_dict[f"multiplier"] = run.summary[f"multiplier/{i}"] if split == "train" else np.nan
                    run_dict["split"] = split
                    run_dict["run_id"] = run.id
                    # Get either Constrained/ or ERM/ from the run name, then append model name.
                    #print("run.name", run.name)
                    #debug if ERM run
                    run_dict["Algorithm"] = f"{run.name.split('/')[0]} {run.config['model']}"
                    run_dict["sweep_id"] = run.sweep.id if run.sweep else ''
                    #print("Algorithm", run_dict["Algorithm"])

                    # Get the experiment tag
                    run_dict["experiment_tag"] = tag_experiment(run)

                    # To better plot constrained vs ERM
                    #TODO this is a hack while I consolidate the tags. 
                    run_dict["type"] = "ERM" if run.config['dual_lr'] == 0 else "Constrained"

                    all_runs.append(run_dict)
    print(f"Fetched {run_counter} runs")
    df = pd.DataFrame(all_runs)
    print(f"Total records: {(df.shape)}")
    print(f"Total runs: {df.run_id.nunique()}")
    return df
df = runs_into_df(runs)

100%|██████████| 27/27 [00:00<00:00, 114.28it/s]


Fetched 27 runs
Total records: (16848, 76)
Total runs: 27


In [3]:
df[['run_id',"sweep_id",'Algorithm',"type",'model','data_path','constraint_level','pred_len']].drop_duplicates().sort_values(['model',"pred_len","constraint_level"])

,run_id,sweep_id,Algorithm,type,model,data_path,constraint_level,pred_len
1584,6huhz8hi,bbaceni3,Electricity-StatInformed-ERM-10e-PatchTST Patc...,ERM,PatchTST,electricity.csv,-1,96
3456,dynyggxw,biyyjk6k,Electricity-StatInformed-ERM-10e-PatchTST Patc...,ERM,PatchTST,electricity.csv,-1,96
5328,96v7155v,xt1o0k3o,Electricity-StatInformed-ERM-10e-PatchTST Patc...,ERM,PatchTST,electricity.csv,-1,96
7200,1shx0uhf,uiygf47f,Exchange-StatInformed-ERM-10e-PatchTST PatchTST,ERM,PatchTST,exchange_rate.csv,-1,96
9072,1fe07y92,cm4xo06b,Exchange-StatInformed-ERM-10e-PatchTST PatchTST,ERM,PatchTST,exchange_rate.csv,-1,96
10944,cakr0opd,nt0iu0zo,Exchange-StatInformed-ERM-10e-PatchTST PatchTST,ERM,PatchTST,exchange_rate.csv,-1,96
12816,0marvnec,ipy2y0z8,Weather-StatInformed-ERM-10e-PatchTST PatchTST,ERM,PatchTST,weather.csv,-1,96
14688,0syo5nd7,p675zmi3,Weather-StatInformed-ERM-10e-PatchTST PatchTST,ERM,PatchTST,weather.csv,-1,96
16560,e8slxnqi,44fzvh2o,Weather-StatInformed-ERM-10e-PatchTST PatchTST,ERM,PatchTST,weather.csv,-1,96
1008,l9y5x65a,bbaceni3,Electricity-StatInformed-ERM-10e-PatchTST Patc...,ERM,PatchTST,electricity.csv,-1,192


# Get IQR for all models, and pred lengths, datasets
Note that the IQRs are vastly different for autoformer and reformer.

# Validation percentiles

In [4]:
stats=df.query("split=='val'").groupby(['data_path','model','pred_len'])['mse'].describe().reset_index()
stats[['data_path','model','pred_len','25%','50%','75%','mean','std']].sort_values(['data_path','pred_len','model'])

,data_path,model,pred_len,25%,50%,75%,mean,std
0,electricity.csv,PatchTST,96,0.150521,0.166091,0.194463,0.171694,0.025739
1,electricity.csv,PatchTST,192,0.151091,0.159778,0.195087,0.171584,0.027561
2,electricity.csv,PatchTST,336,0.155649,0.192292,0.203021,0.187296,0.032424
3,exchange_rate.csv,PatchTST,96,0.075133,0.140187,0.194148,0.134256,0.070453
4,exchange_rate.csv,PatchTST,192,0.139551,0.267443,0.323748,0.232224,0.108221
5,exchange_rate.csv,PatchTST,336,0.233887,0.414493,0.589718,0.399188,0.202345
6,weather.csv,PatchTST,96,0.408730,0.487015,0.520378,0.446082,0.101256
7,weather.csv,PatchTST,192,0.496284,0.543580,0.567473,0.521260,0.103844
8,weather.csv,PatchTST,336,0.548244,0.649129,0.692347,0.610982,0.123730


## Train percentiles

In [5]:
stats=df.query("split=='train'").groupby(['data_path','model','pred_len'])['mse'].describe().reset_index()
stats[['data_path','model','pred_len','25%','50%','75%','mean','std']].sort_values(['data_path','pred_len','model'])

,data_path,model,pred_len,25%,50%,75%,mean,std
0,electricity.csv,PatchTST,96,0.190831,0.201316,0.236842,0.209976,0.029740
1,electricity.csv,PatchTST,192,0.193083,0.198812,0.237168,0.211074,0.029308
2,electricity.csv,PatchTST,336,0.197333,0.243364,0.251142,0.233136,0.035640
3,exchange_rate.csv,PatchTST,96,0.060047,0.118446,0.180120,0.121112,0.070473
4,exchange_rate.csv,PatchTST,192,0.119990,0.250608,0.369855,0.245610,0.141070
5,exchange_rate.csv,PatchTST,336,0.219764,0.432196,0.629846,0.422495,0.236933
6,weather.csv,PatchTST,96,0.441969,0.522916,0.549650,0.479260,0.095870
7,weather.csv,PatchTST,192,0.523938,0.554709,0.580476,0.541969,0.098249
8,weather.csv,PatchTST,336,0.555161,0.663465,0.692575,0.614026,0.110440
